In [180]:
import pandas as pd


import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio



In [181]:
df = pd.read_csv('dataset_social_technical_1d.csv')

In [182]:


# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe" # to be replaced by "iframe" if working on JULIE

target_variable = ['price_pct_variation']
num_features = df.loc[:, df.columns != 'date'].columns

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')


/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '13/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '16/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '17/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '18/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '19/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/tmp/ipykernel_102/715521829.py:11: UserWarning:

Parsing '20/01/2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141 entries, 21 to 1127
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  1141 non-null   datetime64[ns]
 1   SMA_ratio             1141 non-null   float64       
 2   SMA_Volume_Ratio      1141 non-null   float64       
 3   Stochastic_15         1141 non-null   float64       
 4   Stochastic_Ratio      1141 non-null   float64       
 5   RSI_15                1141 non-null   float64       
 6   RSI_ratio             1141 non-null   float64       
 7   MACD                  1141 non-null   float64       
 8   price_pct_variation   1141 non-null   float64       
 9   title_vader_compound  1141 non-null   float64       
 10  title_roberta_neg     1141 non-null   float64       
 11  title_roberta_neu     1141 non-null   float64       
 12  title_roberta_pos     1141 non-null   float64       
dtypes: datetime64[ns]

In [184]:
df = df.sort_values(by='date')


In [185]:
df.head()


,date,SMA_ratio,SMA_Volume_Ratio,Stochastic_15,Stochastic_Ratio,RSI_15,RSI_ratio,MACD,price_pct_variation,title_vader_compound,title_roberta_neg,title_roberta_neu,title_roberta_pos
21,2017-01-02,1.007869,1.212012,79.216907,0.973348,53.144279,1.181372,0.213606,0.016765,0.492067,0.022884,0.839677,0.137439
41,2017-01-03,0.986541,0.735280,95.843722,0.865634,81.615121,0.879563,-0.888218,0.051841,0.068420,0.324355,0.480714,0.194931
93,2017-01-06,0.985472,0.813459,97.567344,1.112130,67.098117,1.397408,-1.238980,0.019584,0.118850,0.022141,0.613193,0.364667
150,2017-01-09,0.999514,1.263688,88.010179,0.872260,67.523111,0.954962,-0.521968,0.005182,0.250333,0.187796,0.389466,0.422738
190,2017-01-11,1.002497,1.259778,63.583964,0.847431,57.815825,0.950078,-0.618205,0.011603,0.000000,0.080081,0.848663,0.071256


In [186]:

# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X = df[features_list]
y = df[target_variable]

X_train = X[:800]
X_test = X[800:]
y_train = y[:800]
y_test = y[800:]

# X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("...Done.")
print()



Dividing into train and test sets...
...Done.



In [187]:
X_train.head()

,SMA_ratio,SMA_Volume_Ratio,Stochastic_15,Stochastic_Ratio,RSI_15,RSI_ratio,MACD,price_pct_variation,title_vader_compound,title_roberta_neg,title_roberta_neu,title_roberta_pos
21,1.007869,1.212012,79.216907,0.973348,53.144279,1.181372,0.213606,0.016765,0.492067,0.022884,0.839677,0.137439
41,0.986541,0.735280,95.843722,0.865634,81.615121,0.879563,-0.888218,0.051841,0.068420,0.324355,0.480714,0.194931
93,0.985472,0.813459,97.567344,1.112130,67.098117,1.397408,-1.238980,0.019584,0.118850,0.022141,0.613193,0.364667
150,0.999514,1.263688,88.010179,0.872260,67.523111,0.954962,-0.521968,0.005182,0.250333,0.187796,0.389466,0.422738
190,1.002497,1.259778,63.583964,0.847431,57.815825,0.950078,-0.618205,0.011603,0.000000,0.080081,0.848663,0.071256


In [188]:
y_train.head()

,price_pct_variation
21,0.016765
41,0.051841
93,0.019584
150,0.005182
190,0.011603


In [189]:
X_test.head()

,SMA_ratio,SMA_Volume_Ratio,Stochastic_15,Stochastic_Ratio,RSI_15,RSI_ratio,MACD,price_pct_variation,title_vader_compound,title_roberta_neg,title_roberta_neu,title_roberta_pos
769,1.004580,0.846927,42.791798,1.219027,44.804344,1.624713,-0.219942,-0.002399,0.078000,0.271222,0.617308,0.111470
787,1.043500,0.906688,9.981840,1.824662,11.710302,2.235168,3.050953,-0.017327,-0.040835,0.353675,0.473523,0.172802
819,1.023939,1.086504,0.000000,0.365188,20.196046,0.457472,1.371461,-0.027742,0.104275,0.270984,0.637354,0.091662
842,0.996993,0.660650,47.126425,0.497925,65.849061,0.214032,0.471819,0.006272,0.059015,0.271027,0.630750,0.098223
885,1.036593,0.963401,11.031153,0.443922,25.382243,0.704819,1.990818,0.015342,0.133316,0.350823,0.456947,0.192230


In [190]:
y_test.head()

,price_pct_variation
769,-0.002399
787,-0.017327
819,-0.027742
842,0.006272
885,0.015342


In [191]:
X.shape

(1141, 12)

In [192]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=150, random_state=1, min_samples_leaf=2)

In [193]:
rf.fit(X_train, y_train)

/tmp/ipykernel_102/2725846524.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(min_samples_leaf=2, n_estimators=150, random_state=1)

In [194]:
price_pred_rf = rf.predict(X_test)

In [195]:
y_test.shape

(341, 1)

In [196]:
y_test.head()

,price_pct_variation
769,-0.002399
787,-0.017327
819,-0.027742
842,0.006272
885,0.015342


In [197]:
type(price_pred_rf)

numpy.ndarray

In [198]:
# Calculate the absolute errors and MAPE
errors_rf = abs(price_pred_rf - y_test['price_pct_variation'])

In [199]:
errors_rf

769     0.000054
787     0.000048
819     0.000178
842     0.000042
885     0.000016
          ...   
1126    0.000057
1000    0.000010
1064    0.000037
1084    0.000059
1127    0.000002
Name: price_pct_variation, Length: 341, dtype: float64

In [200]:
mape_rf = 100 * (errors_rf / y_test['price_pct_variation'])

In [201]:
mape_rf

769    -2.248400
787    -0.276126
819    -0.641942
842     0.665522
885     0.106714
          ...   
1126   -0.520597
1000    0.122923
1064    4.820517
1084    0.359809
1127   -0.010406
Name: price_pct_variation, Length: 341, dtype: float64

In [202]:
mae_rf = round(np.mean(errors_rf), 2)

In [203]:
mae_rf

0.0

In [204]:
accuracy_rf = 100 - np.mean(mape_rf)


In [205]:
type(mape_rf)

pandas.core.series.Series

In [206]:
mape_rf[0:10]

769   -2.248400
787   -0.276126
819   -0.641942
842    0.665522
885    0.106714
928    1.073870
951    5.485622
805   -0.057023
806    1.191011
807   -0.042337
Name: price_pct_variation, dtype: float64

In [207]:
from sklearn.metrics import r2_score, f1_score, ConfusionMatrixDisplay, RocCurveDisplay

In [209]:
# Perform grid search
print("Grid search...")
rf_regressor = RandomForestRegressor()

# Grid of values to be tested
params = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 30]
}
gridsearch = GridSearchCV(rf_regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)


Grid search...


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/s

...Done.
Best hyperparameters :  {'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10}
Best validation accuracy :  0.975287237263767


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:926: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Grid search...


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/s

...Done.
Best hyperparameters :  {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 100}
Best validation accuracy :  0.28381848625891465


In [210]:
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation accuracy : ", gridsearch.best_score_)

Best hyperparameters :  {'max_depth': 6, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10}
Best validation accuracy :  0.975287237263767


In [211]:
features_list

Index(['SMA_ratio', 'SMA_Volume_Ratio', 'Stochastic_15', 'Stochastic_Ratio',
       'RSI_15', 'RSI_ratio', 'MACD', 'price_pct_variation',
       'title_vader_compound', 'title_roberta_neg', 'title_roberta_neu',
       'title_roberta_pos'],
      dtype='object')

In [212]:
target_variable

['price_pct_variation']

In [214]:
vader_features = list(features_list)
vader_features.remove('title_roberta_neg')
vader_features.remove('title_roberta_neu')
vader_features.remove('title_roberta_pos')

In [215]:
vader_features

['SMA_ratio',
 'SMA_Volume_Ratio',
 'Stochastic_15',
 'Stochastic_Ratio',
 'RSI_15',
 'RSI_ratio',
 'MACD',
 'price_pct_variation',
 'title_vader_compound']

In [216]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X = df[vader_features]
y = df[target_variable]

X_train = X[:800]
X_test = X[800:]
y_train = y[:800]
y_test = y[800:]

# X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("...Done.")
print()


Dividing into train and test sets...
...Done.



In [217]:
rf.fit(X_train, y_train)


/tmp/ipykernel_102/3789948204.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(min_samples_leaf=2, n_estimators=150, random_state=1)

In [218]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)


In [219]:
r2_score(y_train_pred, y_train)

0.9914998713737574

In [220]:
r2_score(y_test_pred, y_test)

0.9960544050174618

In [221]:

# Grid of values to be tested
params = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 30]
}
gridsearch = GridSearchCV(rf_regressor, param_grid = params, cv = 3, verbose=0) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation score : ", gridsearch.best_score_)


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/s

...Done.
Best hyperparameters :  {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 30}
Best validation accuracy :  0.9753809667045799


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:926: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [222]:
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation score : ", gridsearch.best_score_)

Best hyperparameters :  {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 30}
Best validation score :  0.9753809667045799


In [223]:
print("Best validation score : ", gridsearch.best_score_)

Best validation score :  0.9753809667045799


In [224]:
# Same computation with only roberta values
target_variable = ['price_pct_variation']
num_features = df.loc[:, df.columns != 'date'].columns
roberta_features_list = list(num_features)
roberta_features_list

['SMA_ratio',
 'SMA_Volume_Ratio',
 'Stochastic_15',
 'Stochastic_Ratio',
 'RSI_15',
 'RSI_ratio',
 'MACD',
 'price_pct_variation',
 'title_vader_compound',
 'title_roberta_neg',
 'title_roberta_neu',
 'title_roberta_pos']

In [225]:
roberta_features_list.remove('price_pct_variation')
roberta_features_list.remove('title_vader_compound')


In [226]:
roberta_features_list

['SMA_ratio',
 'SMA_Volume_Ratio',
 'Stochastic_15',
 'Stochastic_Ratio',
 'RSI_15',
 'RSI_ratio',
 'MACD',
 'title_roberta_neg',
 'title_roberta_neu',
 'title_roberta_pos']

In [227]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X = df[roberta_features_list]
y = df[target_variable]

X_train = X[:800]
X_test = X[800:]
y_train = y[:800]
y_test = y[800:]

# X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("...Done.")
print()


Dividing into train and test sets...
...Done.



In [228]:
rf.fit(X_train, y_train)

/tmp/ipykernel_102/2725846524.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(min_samples_leaf=2, n_estimators=150, random_state=1)

In [229]:

# Grid of values to be tested
params = {
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [1, 2, 5],
    'min_samples_split': [2, 4, 8],
    'n_estimators': [10, 20, 30]
}
gridsearch = GridSearchCV(rf_regressor, param_grid = params, cv = 3, verbose=0) # cv : the number of folds to be used for CV
gridsearch.fit(X_train, y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearch.best_params_)
print("Best validation score : ", gridsearch.best_score_)


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/opt/conda/lib/python3.9/site-packages/s

...Done.
Best hyperparameters :  {'max_depth': 6, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 30}
Best validation score :  0.28770346857102075


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_search.py:926: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [230]:
# Predictions on training set
print("Predictions on training set...")
y_train_pred = gridsearch.predict(X_train)

print("Predictions on test set...")
y_test_pred = gridsearch.predict(X_test)
print("...Done.")
#print(y_train_pred)
print()



Predictions on training set...
Predictions on test set...
...Done.



In [231]:
y_train.head()

,price_pct_variation
21,0.016765
41,0.051841
93,0.019584
150,0.005182
190,0.011603


In [232]:
pred = np.array(y_train_pred)
expected = np.array(y_train['price_pct_variation'])
pred = np.array(y_test_pred)
expected = np.array(y_test['price_pct_variation'])
print("r2 score = ", r2_score(expected, pred))
print()


r2 score =  0.3562524465476289

